<a href="https://colab.research.google.com/github/ALejandroCh17/Tennis-Ball-Robot/blob/daniel_branch/model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
base_dir = '/content/drive/My Drive/capstone_dataset_4/'

train_dir = '/content/drive/My Drive/capstone_dataset_4/train'
test_dir = '/content/drive/My Drive/capstone_dataset_4/test'
valid_dir = '/content/drive/My Drive/capstone_dataset_4/valid'

In [14]:
import os

trainFileCount = len([name for name in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, name))])
testFileCount = len([name for name in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, name))])
validFileCount = len([name for name in os.listdir(valid_dir) if os.path.isfile(os.path.join(valid_dir, name))])

print("Train count: ", str(trainFileCount))
print("Test count: ", str(testFileCount))
print("Valid count: ", str(validFileCount))
print("Total: ", str(trainFileCount+testFileCount+validFileCount))

Train count:  2241
Test count:  104
Valid count:  207
Total:  2552


In [ ]:
#combining all the data
import shutil

#create new directory to store total set
totalDir = '/content/drive/My Drive/capstone_dataset_4/combined'
os.makedirs(totalDir, exist_ok=True)

#iterate over all 3 directories and copy files to totalDir
for directory in [train_dir, test_dir, valid_dir]:
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            shutil.copy2(os.path.join(directory, filename), os.path.join(totalDir, filename))

In [16]:
#count the number of files in the new total directory
totalFileCount = len([name for name in os.listdir(totalDir) if os.path.isfile(os.path.join(totalDir, name))])

print("Total combined count: ", str(totalFileCount))

Total combined count:  2549


Removed 3 non images files, the JSON files

In [30]:
# prompt: set up data preprocessing and augmentation pipelines for totalDir for one class classification

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the image data generator with augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the data from the combined directory
dataset = datagen.flow_from_directory(
    totalDir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 0 images belonging to 0 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(640, 640, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(5, activation='linear')  # The output layer has 5 nodes corresponding to the class label and bounding box coordinates
])

In [6]:
model.compile(optimizer=Adam(lr=1e-4),
              loss='mean_squared_error',
              metrics=['accuracy'])

In [7]:
import os
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array

class BoundingBoxDataGenerator(Sequence):
    def __init__(self, images_dir, labels_dir, batch_size, img_size, shuffle=True):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.image_filenames = os.listdir(images_dir)
        if shuffle:
            np.random.shuffle(self.image_filenames)

    def __len__(self):
        # Compute the number of batches to produce
        return int(np.floor(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.image_filenames[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [k for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        # Updates indexes after each epoch
        if self.shuffle == True:
            np.random.shuffle(self.image_filenames)

    def __data_generation(self, list_IDs_temp):
        # Initialization
        X = np.empty((self.batch_size, *self.img_size, 3))
        y = np.empty((self.batch_size, 5))  # Assuming one bounding box per image; adjust if more

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img_path = os.path.join(self.images_dir, ID)
            img = load_img(img_path, target_size=self.img_size)
            X[i,] = img_to_array(img) / 255.0

            # Store class label and bounding box
            label_path = os.path.join(self.labels_dir, ID.replace('.jpg', '.txt'))
            with open(label_path, 'r') as file:
                bounding_box = np.loadtxt(file)
                y[i,] = bounding_box[0] if bounding_box.ndim == 1 else bounding_box[0, :]  # Adjust if more bounding boxes

        return X, y

In [8]:
train_gen = BoundingBoxDataGenerator(
    images_dir=train_images_dir,
    labels_dir=train_labels_dir,
    batch_size=32,
    img_size=(640, 640)  # Adjust based on your needs
)

In [9]:
validation_generator = BoundingBoxDataGenerator(
    images_dir=valid_images_dir,
    labels_dir=valid_labels_dir,
    batch_size=32,
    img_size=(640, 640)  # Replace with the size you're using for your images
)

In [10]:
model.fit(train_gen, epochs=10, validation_data=validation_generator)

KeyboardInterrupt: 